
---

## **1. ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW**

* **Rule:** Include **all rows from the start of the partition up to the current row** (physical rows).
* **Use:** Cumulative sum by physical order.

**Query:**

```sql
SUM(sales) OVER(PARTITION BY department ORDER BY day ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
```

**Step-by-step (Department A):**

| id | day | sales | Calculation         | running\_total |
| -- | --- | ----- | ------------------- | -------------- |
| 1  | 1   | 100   | 100                 | 100            |
| 2  | 2   | 200   | 100+200             | 300            |
| 3  | 3   | 300   | 100+200+300         | 600            |
| 4  | 4   | 400   | 100+200+300+400     | 1000           |
| 10 | 5   | 500   | 100+200+300+400+500 | 1500           |

* **Department B:** Same logic.

✅ This is **classic cumulative sum**.

---

## **2. ROWS BETWEEN 2 PRECEDING AND CURRENT ROW**

* **Rule:** Include **current row + 2 rows before** (max 3 rows).
* **Use:** Rolling sum over 3 physical rows.

**Step-by-step (Department A):**

| id | day | sales | Calculation | rolling\_3\_rows |
| -- | --- | ----- | ----------- | ---------------- |
| 1  | 1   | 100   | 100         | 100              |
| 2  | 2   | 200   | 100+200     | 300              |
| 3  | 3   | 300   | 100+200+300 | 600              |
| 4  | 4   | 400   | 200+300+400 | 900              |
| 10 | 5   | 500   | 300+400+500 | 1200             |

* **Notice:** Only **previous 2 rows + current** are summed.

---

## **3. ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING**

* **Rule:** Include **one row before, current row, one row after**.
* **Use:** Centered 3-row sum.

**Step-by-step (Department A):**

| id | day | sales | Included rows | centered\_3\_rows |
| -- | --- | ----- | ------------- | ----------------- |
| 1  | 1   | 100   | 100+200       | 300               |
| 2  | 2   | 200   | 100+200+300   | 600               |
| 3  | 3   | 300   | 200+300+400   | 900               |
| 4  | 4   | 400   | 300+400+500   | 1200              |
| 10 | 5   | 500   | 400+500       | 900               |

* **Notice:**

  * First and last rows don’t have a previous or next row, so fewer rows included.
  * Always a **3-row window if possible**.

---

## **4. ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING**

* **Rule:** Include **all rows in the partition**.
* **Use:** Sum repeated for each row (like GROUP BY sum).

**Step-by-step (Department A):**

| id | day | sales | Included rows       | total\_department |
| -- | --- | ----- | ------------------- | ----------------- |
| 1  | 1   | 100   | 100+200+300+400+500 | 1500              |
| 2  | 2   | 200   | 100+200+300+400+500 | 1500              |
| 3  | 3   | 300   | 100+200+300+400+500 | 1500              |
| 4  | 4   | 400   | 100+200+300+400+500 | 1500              |
| 10 | 5   | 500   | 100+200+300+400+500 | 1500              |

* **Notice:** Every row in a partition **has the same sum**.

---

## **5. RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW**

* **Rule:** Cumulative sum **by ORDER BY value**, includes **all rows with same ORDER BY value** (not physical rows).

**Step-by-step (Department A, by sales):**

| id | sales | Included rows (sales ≤ current) | range\_cumulative |
| -- | ----- | ------------------------------- | ----------------- |
| 1  | 100   | 100                             | 100               |
| 2  | 200   | 100,200                         | 300               |
| 3  | 300   | 100,200,300                     | 600               |
| 4  | 400   | 100,200,300,400                 | 1000              |
| 10 | 500   | 100,200,300,400,500             | 1500              |

* **Notice:** If two rows had **same sales**, both included in cumulative.

---

## **6. ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING**

* **Rule:** Include **current row + all rows after**.
* **Use:** Reverse cumulative sum.

**Step-by-step (Department A):**

| id | day | sales | Included rows       | reverse\_cumulative |
| -- | --- | ----- | ------------------- | ------------------- |
| 1  | 1   | 100   | 100+200+300+400+500 | 1500                |
| 2  | 2   | 200   | 200+300+400+500     | 1400                |
| 3  | 3   | 300   | 300+400+500         | 1200                |
| 4  | 4   | 400   | 400+500             | 900                 |
| 10 | 5   | 500   | 500                 | 500                 |

* **Notice:**

  * Think of it as **cumulative sum from bottom to top**.

---

✅ Now you have:

* **Exact rows included per function**
* **Step-by-step calculations**
* **Outputs fully explained**

---


### RANGE depends on **ORDER BY values**, not physical rows. Let's see a **better scenario** to explain it clearly.

---

## **Better Scenario Table for RANGE**

| id | department | sales |
| -- | ---------- | ----- |
| 1  | A          | 100   |
| 2  | A          | 200   |
| 3  | A          | 200   |
| 4  | A          | 300   |
| 5  | A          | 400   |

**Notice:** There are **duplicate sales values (200)**.

---

### **Query**

```sql
SELECT 
  id, department, sales,
  SUM(sales) OVER(
      PARTITION BY department 
      ORDER BY sales 
      RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS range_cumulative
FROM sales;
```

---

### **Step-by-step Explanation**

* **ORDER BY sales:** We're ordering by the **value of sales**, not by row position.
* **RANGE UNBOUNDED PRECEDING AND CURRENT ROW:** Include **all rows with `sales <= current row's sales`**.
* **Important:** All **peers with the same sales value** are included together.

---

| id | sales | Included sales (RANGE ≤ current) | range\_cumulative |
| -- | ----- | -------------------------------- | ----------------- |
| 1  | 100   | 100                              | 100               |
| 2  | 200   | 100,200,200                      | 500               |
| 3  | 200   | 100,200,200                      | 500               |
| 4  | 300   | 100,200,200,300                  | 800               |
| 5  | 400   | 100,200,200,300,400              | 1200              |

✅ **Key Takeaways:**

1. For **id 2 and 3**, even though they are two separate rows, they are **“peers” with same ORDER BY value (sales = 200)**.
2. RANGE **includes all rows with sales ≤ current row**, so **both rows with 200 are included**.
3. This is different from **ROWS**, which would strictly take **physical preceding rows**.

---